In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display, Markdown

# 1. The New Way - Docker Deploy

To start, ensure you have Docker installed and running!

In your terminal, run:
```
docker desktop enable model-runner --tcp=12434
```

Then run 
```
docker model pull ai/qwen3:8B-Q4_0
```
to pull a quantized qwen3 8b model.

In [3]:
docker_url = "http://localhost:12434/engines/v1"
docker_llm = "ai/qwen3:8B-Q4_0"

#### Testing the Docker deployed model - with OpenAI!

In [4]:
from openai import OpenAI

docker_client = OpenAI(base_url=docker_url, api_key="fake")

prompt = "Why is the sky blue?"
response = docker_client.chat.completions.create(
    model=docker_llm,
    messages = [
        {"role": "user", "content": prompt}
    ]
)

In [5]:
display(Markdown(response.choices[0].message.content))

The sky appears blue due to a phenomenon called **Rayleigh scattering**, which involves how sunlight interacts with the Earth's atmosphere. Here's a step-by-step explanation:

1. **Sunlight Composition**: Sunlight is a mix of all visible colors (violet, blue, green, yellow, orange, red), each with different wavelengths. Blue and violet have shorter wavelengths, while red and orange have longer ones.

2. **Atmospheric Interaction**: When sunlight enters Earth's atmosphere, it collides with molecules (like nitrogen and oxygen) and small particles. These particles scatter light in all directions.

3. **Rayleigh Scattering**: Shorter wavelengths (blue/violet) scatter more efficiently than longer ones. This is because the scattering is inversely proportional to the fourth power of the wavelength (Rayleigh scattering). As a result, blue light is scattered more widely than red or yellow light.

4. **Human Eye Sensitivity**: While violet light scatters even more than blue, the human eye is less sensitive to violet, and some violet light is absorbed by the upper atmosphere. This combination makes the sky appear **blue** rather than violet.

5. **Sunrise/Sunset Effect**: During sunrise or sunset, sunlight travels through more atmosphere, scattering out the blue light and leaving the longer wavelengths (red, orange) to dominate, creating the warm hues.

**Key Takeaway**: The sky is blue because the atmosphere scatters shorter blue wavelengths more effectively, and our eyes perceive this scattered blue light as the dominant color.

#### Using LangChain

In [6]:
from langchain_openai import ChatOpenAI

langchain_llm = ChatOpenAI(
    model_name = docker_llm,
    temperature = 0,
    api_key = "fake",
    openai_api_base = docker_url
)

response = langchain_llm.invoke([{"role": "user", "content": prompt}])

In [7]:
display(Markdown(response.content))

The sky appears blue due to a phenomenon called **Rayleigh scattering**, which involves how sunlight interacts with Earth's atmosphere. Here's a step-by-step explanation:

1. **Sunlight Composition**:  
   Sunlight is a mix of all visible colors (violet, blue, green, yellow, orange, red), each with different **wavelengths**. Blue and violet have shorter wavelengths, while red and orange have longer ones.

2. **Atmospheric Interaction**:  
   When sunlight enters Earth's atmosphere, it collides with molecules (like nitrogen and oxygen) and small particles. These scatter the light in all directions. **Shorter wavelengths (blue/violet) scatter more easily** than longer ones (red/orange), a process known as **Rayleigh scattering**.

3. **Why Blue Dominates**:  
   - **Blue light** (shorter wavelength) scatters more than red, making it spread throughout the sky.  
   - **Violet light** (even shorter wavelength) scatters even more, but our eyes are less sensitive to violet, and some of it is absorbed or scattered by the atmosphere.  
   - The combination of scattered blue and our eyes' sensitivity makes the sky appear **blue** to us.

4. **Sunset and Sunrise Colors**:  
   During sunrise or sunset, sunlight travels a longer path through the atmosphere. This scatters away most of the blue light, leaving **red and orange wavelengths** to dominate, creating the warm hues we see.

5. **Contrast with the Ocean**:  
   The ocean appears blue because water absorbs longer wavelengths (red/orange) and reflects blue light. The sky's blue is purely due to atmospheric scattering, not water.

**In Summary**: The sky is blue because shorter-wavelength blue light is scattered more efficiently by atmospheric molecules, and our eyes perceive this scattered light as blue. The absence of violet in the sky is due to both physical scattering and human eye sensitivity.

#### Using LlamaIndex

In [8]:
import nest_asyncio
from llama_index.llms.openai_like import OpenAILike

nest_asyncio.apply()

llm = OpenAILike(
    model = docker_llm,
    api_base = docker_url,
    api_key = "fake",
    is_chat_model = True,
    is_function_calling_model = True,
    temperature = 0
)

response = llm.complete("Hi!")

display(Markdown(str(response)))

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Hi! 😊 How can I assist you today? Whether you have questions, need help with something, or just want to chat, I'm here for you! What's on your mind?

Creating a tool

In [9]:
from llama_index.core.agent.workflow import AgentStream, FunctionAgent
from llama_index.core.tools import FunctionTool

def get_weather(location: str) -> str:
    """Use this tool to get the weather in a location."""
    return f"The weather in {location} is sunny."

weather_agent = FunctionAgent(
    llm = llm,
    tools = [FunctionTool.from_defaults(get_weather)]
)

In [10]:
handler = weather_agent.run("Give me the weather in Sydney?")

async for event in handler.stream_events():
    if isinstance(event, AgentStream):
        print(event.delta, end="", flush=True)

The weather in Sydney is currently sunny. Enjoy your time outdoors! 🌞

# 2. My favorite way - Ollama!

To start, ensure that you have Ollama installed and running.

In your terminal, run:
```
ollama pull qwen2.5
```

This will pull qwen2.5:7b by default

In [19]:
ollama_url = "http://localhost:11434/v1/"
ollama_llm = "qwen3"

#### Using the OpenAI SDK

In [20]:
ollama_client = OpenAI(base_url=ollama_url, api_key="fake")

response = ollama_client.chat.completions.create(
    model=ollama_llm,
    messages = [
        {"role": "user", "content": prompt}
    ]
)

display(Markdown(response.choices[0].message.content))

<think>
Okay, so the user is asking why the sky is blue. I remember this is a classic question. Let me think about how to explain it. I think it has to do with the way light interacts with the atmosphere. 

First, sunlight is white, but it's made up of different colors, each with different wavelengths. I think the atmosphere scatters light, and blue light has a shorter wavelength, so it gets scattered more than other colors. That's Rayleigh scattering, right? 

Wait, but why does that make the sky blue instead of violet? Because violet has an even shorter wavelength, so why isn't the sky violet? Maybe because our eyes are more sensitive to blue, and there's more blue light in the sunlight. Plus, some of the violet light is absorbed by the atmosphere. 

Also, during sunrise or sunset, the sky turns red or orange. That's because the sunlight has to pass through more atmosphere, so the blue light is scattered away, leaving the longer wavelengths like red and orange. 

I should check if there are any other factors. Maybe the presence of molecules in the atmosphere, like nitrogen and oxygen, are responsible for the scattering. Also, the density of the atmosphere affects how much scattering happens. 

Wait, but I should make sure I'm not confusing Rayleigh scattering with Mie scattering. Mie scattering is for larger particles, like water droplets, which causes white clouds and the white color. So the blue sky is due to Rayleigh scattering from small gas molecules. 

I need to explain this in a simple way. Start with sunlight, different colors, scattering, blue is scattered more, but why not violet. Also, mention the human eye's sensitivity. Then explain sunrise/sunset as a side note. 

Let me structure the answer step by step: first, sunlight's spectrum, then scattering in the atmosphere, why blue and not violet, and the effect of sun position. Maybe also mention that without atmosphere, the sky would be black. That adds context. 

Wait, but should I mention the role of nitrogen and oxygen specifically? Or just say air molecules? Maybe the latter is simpler. Also, the fact that blue photons are scattered in all directions, which is why the sky appears blue from all directions, not just directly above. 

I think that's a good outline. Make sure it's clear and not too technical, but accurate. Let me put that together now.
</think>

The sky appears blue due to a phenomenon called **Rayleigh scattering**, which involves how sunlight interacts with the Earth's atmosphere. Here's a simplified explanation:

1. **Sunlight and the Spectrum**:  
   Sunlight is white, but it consists of a spectrum of colors (like a rainbow), each with different wavelengths (violet, blue, green, yellow, orange, red). Shorter wavelengths (like blue and violet) scatter more easily than longer ones (like red or yellow) when light passes through a medium.

2. **Atmospheric Scattering**:  
   The Earth's atmosphere is filled with molecules (like nitrogen and oxygen). When sunlight enters the atmosphere, the shorter-wavelength blue and violet light is scattered in all directions by these molecules. This scattering is called **Rayleigh scattering** and is inversely proportional to the fourth power of the wavelength (shorter wavelengths scatter more).

3. **Why Blue, Not Violet?**  
   While violet light has an even shorter wavelength than blue and scatters more, the Sun emits less violet light than blue. Additionally, some violet light is absorbed by the upper atmosphere. Human eyes are also more sensitive to blue light than violet, so we perceive the sky as blue.

4. **Sunrise and Sunset**:  
   During these times, sunlight travels through more of the atmosphere, scattering most of the blue light out of our line of sight. This leaves the longer wavelengths (reds, oranges, yellows) to dominate, creating the warm hues of dawn and dusk.

5. **Without Atmosphere**:  
   If Earth had no atmosphere, the sky would appear black, as there would be no scattering of sunlight to create colors.

In summary, the blue sky is a result of Rayleigh scattering, where blue light is scattered more efficiently by atmospheric molecules, and our eyes perceive this scattered blue light across the entire sky.

#### Using the Ollama SDK

Which looks like the OpenAI SDK!

In [21]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(
    model=ollama_llm, 
    messages=[
        {"role": "user", "content": prompt}
    ]
  )

print(response['message']['content'])
# or access fields directly from the response object
# print(response.message.content)

<think>
Okay, so the user is asking why the sky is blue. I remember from school that it has something to do with sunlight and the atmosphere, but I need to recall the exact details. Let me think... I think it's about the scattering of light. Oh right, Rayleigh scattering. But wait, why does that make the sky blue and not another color? 

First, sunlight is white, but it's made up of different colors, each with different wavelengths. The shorter wavelengths, like blue and violet, scatter more than the longer ones like red or yellow. But why do we see blue instead of violet? Maybe because the sun emits more blue light than violet, and our eyes are more sensitive to blue. Also, some of the violet light is absorbed by the upper atmosphere. 

Wait, but when the sun is low, like during sunrise or sunset, the sky turns red or orange. That's because the light has to pass through more atmosphere, so the blue light gets scattered away, leaving the longer wavelengths. So that's the reason for the

#### Using LangChain

In [22]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model = ollama_llm,
    temperature = 0
)

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to Chinese. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = model.invoke(messages)

In [23]:
display(Markdown(ai_msg.content))

<think>
Okay, the user wants me to translate "I love programming." from English to Chinese. Let me start by understanding the sentence. The main components are "I" (subject), "love" (verb), and "programming" (object). 

First, "I" in Chinese is "我". Then "love" is a verb, and the translation for "love" in this context would be "爱" or "喜欢". Both are correct, but "爱" is more intense, while "喜欢" is a bit more casual. Since the user said "love", maybe "爱" is better here.

Next, "programming" translates to "编程". So putting it all together: "我爱编程。" Alternatively, "我喜欢编程。" Both are correct. But the user might prefer the more direct translation. Let me check if there are any nuances. 

In Chinese, "爱" is often used for deeper affection, while "喜欢" is more about preference. Since the user says "love", using "爱" would be more accurate. So the translation should be "我爱编程。" 

Wait, but sometimes people might use "喜欢" even when they mean "love" in a general sense. Maybe the user is okay with either. But since the instruction is to translate accurately, "爱" is better. 

I should also check for any possible errors. The structure is correct: subject + verb + object. No particles needed. So the final translation is "我爱编程。"
</think>

我爱编程。

In [ ]:
from langgraph.prebuilt import create_react_agent

ollama_langchain_agent = create_react_agent(model, tools=[get_weather])

In [25]:
async for chunk in ollama_langchain_agent.astream(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    stream_mode="updates"
):
    # chunk['messages'].pretty_print()
    print(chunk)

{'agent': {'messages': [AIMessage(content='<think>\nOkay, the user is asking for the weather in SF. SF stands for San Francisco. I need to use the get_weather function. The function requires a location parameter. So I should call get_weather with location set to "San Francisco". Let me make sure there\'s no typo. Yep, that\'s correct. Alright, time to format the tool call.\n</think>\n\n', additional_kwargs={}, response_metadata={'model': 'qwen3', 'created_at': '2025-07-16T22:58:46.38297Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2195141916, 'load_duration': 39597666, 'prompt_eval_count': 147, 'prompt_eval_duration': 397647916, 'eval_count': 97, 'eval_duration': 1757394334, 'model_name': 'qwen3'}, id='run--c34d4c14-98ae-4271-9b90-f454d81e4b3f-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'San Francisco'}, 'id': '8bf42628-8300-4e40-a726-8c4a2c33599d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 147, 'output_tokens': 97, 'total_tokens': 244})]}}
{'